In [7]:
import requests
from bs4 import BeautifulSoup, Comment

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

uni_names = pd.read_csv("./universities.csv", header=None)

uni_names.columns = ['name']

# initialize a pandas dataframe that will contain all the scraped data
# add more columns name as needed
data = pd.DataFrame(columns=['name', "locality", "state", "country", 'url'])

In [14]:
for uni in uni_names.name:
    # make url -> fetch page -> parse html -> select the table on the sidebar
    uni_wiki_url = "https://en.wikipedia.org/wiki/" + "_".join(uni.split(" "))
    page = requests.get(uni_wiki_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    try:
        sidebar_table = soup.find_all('table', {"class": "infobox vcard"})[0]
    except IndexError: # Probably poorly structured page
        continue
    
    # key-value variable to store everything for one particular uni
    juice = {"name" : uni}

    # find url
    try:
        url = sidebar_table.find("span", class_="url")
        url = url.find("a", href=True)
        juice['url'] = url['href']
    except AttributeError: # Can't find university URL
        continue
    
    try:
        queries = ['locality', 'country-name', 'state']
        columns = ['locality', 'country', 'state']
        for query, column in zip(queries, columns):
            location = sidebar_table.find("th", string="Location").parent
            loc = location.find("div", class_=query)
            juice[column] = loc.text
    except:
        # probably, state was missing
        pass
    # Add the new Uni entry to the grand table aka. `data`
    data = data.append(juice, ignore_index=True)  

data

# Write this dataframe to CSV
data.to_csv("uni_data.csv", index=None)

0
Massachusetts Institute of Technology
Harvard University
Stanford University
University of California, Berkeley
Purdue University
Cornell University
University of Washington
Columbia University in the City of New York
University of Michigan
Penn State University
University of Minnesota
University of Wisconsin-Madison
University of California, Los Angeles
New York University
University of Pennsylvania
University of Toronto
Yale University
The University of Texas at Austin
University of Oxford
Princeton University
Carnegie Mellon University
University of Southern California
The University of British Columbia
University of California, Irvine
University of Colorado Boulder
University of Illinois at Urbana-Champaign
University of Cambridge
Arizona State University
University of Chicago
Johns Hopkins University
University of California, San Diego
Michigan State University
The University of Arizona
University of Florida
Rutgers, The State University of New Jersey
Northwestern University
Sha

In [66]:
# url = sidebar_table.find_all("span", class_="url")[0]
# url = url.find("a", href=True)
# url['href']

# th = sidebar_table.find_all("th", string='Location')[0]
# th.parent.td.find('div', class_='locality')

#     lookup_text = ['Location', 'Website']  # name of relevant row in the sidebar on wiki
#     column_names = ['location', 'url']     # corresponding header/column name to be used in DataFrame
    
#     for text, column in zip(lookup_text, column_names):
#         # lookup the info and store it in juice with corresponding column name
#         th = sidebar_table.find_all("th", string=text)[0]
#         juice[column] = th.parent.td.text
        
#         url = sidebar_table.find_all("span", class_="url")[0]
#         url = url.find("a", href=True)
#         juice['url'] = url['href']
    
 
#     # get rid of comments, scripts, styles from html source
#     blacklisted = ["script", "style", "comment"]
#     for tag in soup.findAll():
#             if tag.name.lower() in blacklisted:
#                 tag.extract()


# for tr in sidebar_table.tbody:
#     try:
#         if tr.th.text == 'Website':
#             juice["url"] = tr.td.text
#         elif tr.th.text == 'Location':
#             juice["location"] = tr.td.text
#     except:
#         # empty table row/header
#         pass

In [3]:
uni_ranking_url = "https://www.4icu.org/top-universities-world/"

page = requests.get(uni_ranking_url)
soup_ranking = BeautifulSoup(page.content, 'html.parser')
universities = []
table = soup_ranking.find("table", class_="table table-hover")
for tr in table.find_all('tr')[1:]:
    universities.append(tr.find_all("td")[1].text)

universities = pd.DataFrame(universities)
universities.to_csv("universities.csv", index=None)